In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cpucluster"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        'C': uniform(0.001,10.0),
        'max_iter':uniform(50,100,200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './',
              entry_script='train.py',
             compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 20,
                                    max_concurrent_runs = 4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdriverun = exp.submit(config=hyperdrive_config)
RunDetails(hyperdriverun).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_hdr = hyperdriverun.get_best_run_by_primary_metric()

joblib.dump(best_hdr, 'HyperDriveModel.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=url)

In [ ]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

dataset = pd.concat([x,y],axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5)

In [2]:
# Submit your automl run

experiment = Experiment(ws, "automl_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)

In [1]:
# Retrieve and save your best automl model.

best_automl = automl_run.get_best_child()

joblib.dump(best_automl, 'AutoMLModel.joblib')

NameError: name 'automl_run' is not defined

In [ ]:
cpu_cluster.delete()